En esta segunda parte del archivo voy a hacer el join entre las bases de usuarios ecobici 2024 y la filtrada por los 3 meses, comienzo importando las librerías

In [1]:
import pandas as pd 

Cargo las bases

In [2]:
usua = pd.read_csv("../data/raw/usuarios_ecobici_2024.csv", sep=',', encoding='utf_8')

recorridos_3meses = pd.read_csv("../data/interim/recorridos_3meses.csv", sep=',', encoding='utf_8')

Realizo el join entre estas bases de datos y lo guardo en recorridos_usuarios_3meses

In [3]:
recorridos_usuarios_3meses = usua.merge(recorridos_3meses, on='id_usuario', how='inner')

In [5]:
recorridos_usuarios_3meses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285581 entries, 0 to 285580
Data columns (total 21 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id_usuario                  285581 non-null  int64  
 1   genero_usuario              285581 non-null  object 
 2   edad_usuario                285581 non-null  int64  
 3   fecha_alta                  285581 non-null  object 
 4   hora_alta                   285581 non-null  object 
 5   id_recorrido                285581 non-null  int64  
 6   duracion_recorrido          285581 non-null  int64  
 7   fecha_origen_recorrido      285581 non-null  object 
 8   id_estacion_origen          285581 non-null  int64  
 9   nombre_estacion_origen      285581 non-null  object 
 10  direccion_estacion_origen   285581 non-null  object 
 11  long_estacion_origen        285581 non-null  float64
 12  lat_estacion_origen         285581 non-null  float64
 13  fecha_destino_

Para crear la nueva columna duración_viaje_minutos, que registre el tiempo de los recorridos, teniendo en cuenta las columnas fecha_destino_recorrido - fecha_origen_recorrido, debo convertir esas columnas a tipo datetime

In [6]:
recorridos_usuarios_3meses['fecha_destino_recorrido'] = pd.to_datetime(recorridos_usuarios_3meses['fecha_destino_recorrido'])
recorridos_usuarios_3meses['fecha_origen_recorrido'] = pd.to_datetime(recorridos_usuarios_3meses['fecha_origen_recorrido'])

ahora sí puedo crear la nueva columna, el resultado está en minutos

In [7]:
recorridos_usuarios_3meses['duracion_viaje_minutos'] = ((recorridos_usuarios_3meses['fecha_destino_recorrido'] 
                                                         - recorridos_usuarios_3meses['fecha_origen_recorrido'])
                                                         .dt.total_seconds() / 60)
                                         
recorridos_usuarios_3meses['duracion_viaje_minutos'] = recorridos_usuarios_3meses['duracion_viaje_minutos'].round(2)

Observo que la base de datos original ya tiene una columna duracion_recorrido, expresada en segundos, voy a verificar si son iguales

In [8]:
recorridos_usuarios_3meses[['duracion_viaje_minutos', 'fecha_origen_recorrido', 'fecha_destino_recorrido', 'duracion_recorrido']]

,duracion_viaje_minutos,fecha_origen_recorrido,fecha_destino_recorrido,duracion_recorrido
0,24.22,2024-07-01 09:42:09,2024-07-01 10:06:22,1453
1,31.53,2024-07-01 13:22:10,2024-07-01 13:53:42,1892
2,21.43,2024-07-01 13:40:27,2024-07-01 14:01:53,1286
3,10.22,2024-07-01 14:33:57,2024-07-01 14:44:10,613
4,34.60,2024-07-01 12:22:54,2024-07-01 12:57:30,2076
...,...,...,...,...
285576,5.15,2024-08-13 06:43:12,2024-08-13 06:48:21,309
285577,49.27,2024-08-12 19:03:20,2024-08-12 19:52:36,2956
285578,3.68,2024-08-12 07:57:09,2024-08-12 08:00:50,221
285579,27.27,2024-09-28 12:09:33,2024-09-28 12:36:49,1636


Creo aux_duracion_recorrido, para expresar esa columna en minutos

In [9]:
aux_duracion_recorrido = (recorridos_usuarios_3meses['duracion_recorrido'] / 60).round(2)
aux_duracion_recorrido

0         24.22
1         31.53
2         21.43
3         10.22
4         34.60
          ...  
285576     5.15
285577    49.27
285578     3.68
285579    27.27
285580    47.55
Name: duracion_recorrido, Length: 285581, dtype: float64

In [10]:
print((recorridos_usuarios_3meses['duracion_viaje_minutos'] == aux_duracion_recorrido).value_counts())

True    285581
Name: count, dtype: int64


Observo que el total de las filas son true, por lo tanto los datos concuerdan

Finalmente guardo la base de datos con esa columna creada en recorridos_usuarios_3meses

In [ ]:
ruta_guardado = "data/processed/recorridos_usuarios_3meses.csv"
recorridos_usuarios_3meses.to_csv(ruta_guardado, index=False)